# Cálculo de Cohen d por grupo
Notebook para calcular el tamaño del efecto de Cohen d en variables binarias pre/post por grupo.

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import ipywidgets as widgets
from IPython.display import display, clear_output


In [ ]:
from google.colab import files
uploaded = files.upload()
if uploaded:
    fname = next(iter(uploaded))
    if fname.lower().endswith('.dta'):
        df = pd.read_stata(fname)
    elif fname.lower().endswith(('.xlsx', '.xls')):
        df = pd.read_excel(fname)
    else:
        raise ValueError('Solo se admiten archivos .dta o .xlsx')
    display(df.head())


In [ ]:
def cohend_group(df: pd.DataFrame, varlist: list[str], group: str, method: str = "dz", filename: str = "cohend_resultados.xlsx", sheet_name: str = "resultados") -> pd.DataFrame:
    """Calcula Cohen d para variables binarias pre y post por grupo.
    Devuelve un DataFrame con los resultados y exporta a Excel."""
    if group not in df.columns:
        raise ValueError(f"La columna de grupo {group} no existe en el DataFrame")
    if len(varlist) % 2 != 0:
        raise ValueError("varlist debe tener número par de variables")
    for col in varlist:
        if col not in df.columns:
            raise ValueError(f"La columna {col} no existe en el DataFrame")
    mid=len(varlist)//2
    prevars=varlist[:mid]
    postvars=varlist[mid:]
    df2=df.copy()
    df2[prevars+postvars]=df2[prevars+postvars].apply(pd.to_numeric, errors="coerce")
    df2['tpre']=(df2[prevars]==2).mean(axis=1)
    df2['tpost']=(df2[postvars]==2).mean(axis=1)
    df2['dif']=df2['tpost']-df2['tpre']
    resultados=[]
    for nivel in df2[group].astype(str).unique():
        sub=df2[df2[group].astype(str)==str(nivel)].dropna(subset=['tpre','tpost'])
        N=len(sub)
        if N==0:
            continue
        m_pre=sub['tpre'].mean()
        m_post=sub['tpost'].mean()
        m_diff=sub['dif'].mean()
        sd_pre=sub['tpre'].std(ddof=1)
        sd_post=sub['tpost'].std(ddof=1)
        sd_diff=sub['dif'].std(ddof=1)
        r_pp=sub['tpre'].corr(sub['tpost'])
        if method=='dz':
            denom=sd_diff
        elif method=='dav':
            denom=(sd_pre+sd_post)/2
        elif method=='drm':
            denom=np.sqrt(sd_pre**2+sd_post**2-2*r_pp*sd_pre*sd_post)
        else:
            raise ValueError('method debe ser "dz", "dav" o "drm"')
        d_cohen=m_diff/denom if denom and denom!=0 else np.nan
        if N>=2:
            p_value=stats.ttest_rel(sub['tpost'],sub['tpre'],nan_policy='omit').pvalue
        else:
            p_value=np.nan
        if pd.isna(d_cohen):
            magnitude='NA'
            symbol=''
        elif abs(d_cohen)>=0.80:
            magnitude='GRANDE'
            symbol='★★★'
        elif abs(d_cohen)>=0.50:
            magnitude='MEDIANO'
            symbol='★★'
        elif abs(d_cohen)>=0.20:
            magnitude='PEQUEÑO'
            symbol='★'
        else:
            magnitude='TRIVIAL'
            symbol=''
        resultados.append({'grupo':nivel,'N':N,'m_pre':m_pre,'m_post':m_post,'m_diff':m_diff,'d_cohen':d_cohen,'p_value':p_value,'magnitude':magnitude,'symbol':symbol})
    result_df=pd.DataFrame(resultados)
    titulo=f"Cohen d – Variables: {', '.join(prevars)}, {', '.join(postvars)} (método={method})"
    try:
        import xlsxwriter
        engine='xlsxwriter'
    except ImportError:
        try:
            import openpyxl
            engine='openpyxl'
        except ImportError as e:
            raise ImportError('Necesita instalar "xlsxwriter" u "openpyxl" para exportar a Excel') from e
    with pd.ExcelWriter(filename, engine=engine) as writer:
        result_df.to_excel(writer, sheet_name=sheet_name, startrow=2, index=False)
        ws=writer.sheets[sheet_name]
        if engine=='xlsxwriter':
            ws.write('A1', titulo)
        else:
            ws['A1']=titulo
    print(f'Archivo guardado: {filename}')
    return result_df


In [ ]:
def interfaz_cohend(df):
    num_cols=list(df.select_dtypes(include='number').columns)
    str_cols=list(df.select_dtypes(exclude='number').columns)
    instrucciones=widgets.HTML('<b>Seleccione un número par de variables (mitad PRE, mitad POST). Use el botón Resetear para limpiar la selección.</b>')
    sel_vars=widgets.SelectMultiple(options=num_cols,description='Variables',layout=widgets.Layout(width='50%',height='200px'))
    sel_grupo=widgets.Dropdown(options=str_cols,description='Grupo')
    metodo=widgets.Dropdown(options=['dz','dav','drm'],value='dz',description='Método')
    archivo=widgets.Text(value='cohend_resultados.xlsx',description='Archivo')
    hoja=widgets.Text(value='resultados',description='Hoja')
    btn_calc=widgets.Button(description='Calcular',button_style='primary')
    btn_reset=widgets.Button(description='Resetear',button_style='warning')
    sel_lab=widgets.HTML()
    out=widgets.Output()

    def actualizar_sel(change=None):
        seleccion=list(sel_vars.value)
        if seleccion:
            sel_lab.value=f"<em>Variables seleccionadas:</em> {', '.join(seleccion)}"
        else:
            sel_lab.value='<em>No hay variables seleccionadas.</em>'
    sel_vars.observe(actualizar_sel, names='value')
    actualizar_sel()

    def resetear(b):
        sel_vars.value=tuple()
    btn_reset.on_click(resetear)

    def al_hacer_click(b):
        with out:
            out.clear_output()
            varlist=list(sel_vars.value)
            if len(varlist)==0:
                print('No ha seleccionado variables.')
                return
            if len(varlist)%2!=0:
                print('Seleccione un número par de variables.')
                return
            print(f"Calculando con método {metodo.value} para: {', '.join(varlist)} en grupo {sel_grupo.value}")
            res=cohend_group(df,varlist,group=sel_grupo.value,method=metodo.value,filename=archivo.value,sheet_name=hoja.value)
            display(res)
    btn_calc.on_click(al_hacer_click)
    display(instrucciones,sel_vars,sel_lab,sel_grupo,metodo,archivo,hoja,widgets.HBox([btn_calc,btn_reset]),out)

# Ejecutar después de cargar el DataFrame 'df'
interfaz_cohend(df)
